# CS231n 
## Lecture 02, 03 summary
김진성

1. image classification
2. Nearest Neighbor
3. Linear classifier


4. SVM loss
5. softmax classifier
6. Optimization

---

## Image Classification

Image Classification이란 input으로 이미지 정보를 받으면, 이 이미지가 어떤 이미지인지 분류해주는 행위를 말한다.



~~~ python
def classify_image(image):
    # Some magic here?
    return class_label
~~~

![01.png](img/01.png)


---
### - The Problems & Challenges

computer에서의 image는 semantic한 정보가 아닌, 숫자의 행렬(matrix) 조합으로 구성되어 있다. 따라서 아래와 같은 경우 전혀 다른 숫자의 조합으로 표현되므로 image classification에 어려움을 준다.

+ Viewpoint Variation (보는 각도)

+ Illumincation (조명)

+ Deformation(변형)

+ Occlusion(은폐)

+ Background Clutter(배경과 섞임)

+ Intraclass variation(물체의 다양성)

![02.png](img/02.png)

---
### - Attempts have been made

가장 기초적으로 할 수 있는 방법으로, 규칙을 정하는 것이다.

예를 들어 아래와 같은 고양이의 경우, edge detection을 통해 edge 정보를 얻은 뒤, 귀, 얼굴, 꼬리 등의 존재를 규칙기반으로 판단한 뒤, 고양이 임을 분류하는 방식이다. 이와 같은 방식은 비효율적이므로 사용하지 않는다.


![03.png](img/03.png)

---
### - Data-Driven Approach
##### 1. Collect a dataset of images and labels
![04.png](img/04.png)

##### 2. Use Machine Learning to train a classifier
```python
def train(images, labels):
    # Machine learning
    return model
```

##### 3. Evaluate the classifier on new images
```python
def predict(model, test_images):
    # Use Model to predict labels
    return test_labels
```


---
## Nearest Neighbor
Image classification의 가장 기본적인 접근법이다.

input 이미지가 들어왔을때, 미리 기억된 이미지들과 비교하여, 가장 비슷한 이미지를 선택하고 그 label로 분류하는 방법이다. 

- **train :** Just Memorize all data and labels, 
$O(1)$

- **Predict :** Predict the label of the most similar training image,
$O(N)$

비슷한 이미지를 판별하기 위해, distance metric을 이용한다. distance metric에는 L1 distance와 L2 distance가 존재하며, distance 값이 작을 수록 해당 input 이미지와 비슷하다는 의미이다.

- **Distance metric**
    - L1 (Manhattan) distance
$$d_1(I_1,I_2) = {\displaystyle \sum_{p} |{{I_1^p}-{I_2^p}|  }} $$
    
    - L2 (Euclidean) distance
$$d_2(l_1,l_2) = \sqrt{\displaystyle \sum_{p} {({I_1^p} - {I_2^p})}^2} $$

<details><summary> How to calculate distance with L1 </summary>
    
![05.png](img/05.png)

</details>

---



In [16]:
import numpy as np

def L1_distance(I1,I2):
    return np.sqrt(np.sum(np.abs(I1-I2)))

def L2_distance(I1,I2):
    return np.sqrt(np.sum(np.square(I1,I2)))

In [17]:
class NearestNeighbor(object):
    def __init__(self):
        pass

    def train(self, X, y):
        """ X is N x D where each row is an example. Y is 1-dimension of size N """
        # the nearest neighbor classifier simply remembers all the training data
        self.Xtr = X
        self.ytr = y

    def predict(self, X):
        """ X is N x D where each row is an example we wish to predict label for """
        num_test = X.shape[0]
        # lets make sure that the output type matches the input type
        Ypred = np.zeros(num_test, dtype = self.ytr.dtype)

        # loop over all test rows
        for i in range(num_test):
            # find the nearest training image to the i'th test image
            # using the L1 distance (sum of absolute value differences)
            distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
            min_index = np.argmin(distances) # get the index with smallest distance
            Ypred[i] = self.ytr[min_index] # predict the label of the nearest example

        return Ypred

### - KNN(K-Nearest Neighbor)

KNN은 Nearest Neighbor와 방식은 같으나, predict 과정에서 가장 가까운 이미지 하나를 선택하는 대신, 가장 가까운 K개의 이미지 중 **가장 많은 label (Mjority vote)**을 선택하는 방법이다.


![06.jpg](img/06.jpg)

---

<details><summary>2020 cs231n assignment1 KNN python code <br>
    [assignment1\cs231n\classifiers\k_nearest_neighbor.py]</summary>
    
```python
from builtins import range
from builtins import object
import numpy as np
from past.builtins import xrange


class KNearestNeighbor(object):
    """ a kNN classifier with L2 distance """

    def __init__(self):
        pass

    def train(self, X, y):
        """
        Train the classifier. For k-nearest neighbors this is just
        memorizing the training data.

        Inputs:
        - X: A numpy array of shape (num_train, D) containing the training data
          consisting of num_train samples each of dimension D.
        - y: A numpy array of shape (N,) containing the training labels, where
             y[i] is the label for X[i].
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X, k=1, num_loops=0):
        """
        Predict labels for test data using this classifier.

        Inputs:
        - X: A numpy array of shape (num_test, D) containing test data consisting
             of num_test samples each of dimension D.
        - k: The number of nearest neighbors that vote for the predicted labels.
        - num_loops: Determines which implementation to use to compute distances
          between training points and testing points.

        Returns:
        - y: A numpy array of shape (num_test,) containing predicted labels for the
          test data, where y[i] is the predicted label for the test point X[i].
        """
        if num_loops == 0:
            dists = self.compute_distances_no_loops(X)
        elif num_loops == 1:
            dists = self.compute_distances_one_loop(X)
        elif num_loops == 2:
            dists = self.compute_distances_two_loops(X)
        else:
            raise ValueError('Invalid value %d for num_loops' % num_loops)

        return self.predict_labels(dists, k=k)

    def compute_distances_two_loops(self, X):
        """
        Compute the distance between each test point in X and each training point
        in self.X_train using a nested loop over both the training data and the
        test data.

        Inputs:
        - X: A numpy array of shape (num_test, D) containing test data.

        Returns:
        - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
          is the Euclidean distance between the ith test point and the jth training
          point.
        """
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train))
        for i in range(num_test):
            for j in range(num_train):
                #####################################################################
                # TODO:                                                             #
                # Compute the l2 distance between the ith test point and the jth    #
                # training point, and store the result in dists[i, j]. You should   #
                # not use a loop over dimension, nor use np.linalg.norm().          #
                #####################################################################
                # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

               	dists[i,j] = np.sqrt(np.sum( np.square(X[i,:] - self.X_train[j,:])))

                # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        return dists

    def compute_distances_one_loop(self, X):
        """
        Compute the distance between each test point in X and each training point
        in self.X_train using a single loop over the test data.
		
        Input / Output: Same as compute_distances_two_loops
        """
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train))
        for i in range(num_test):
            #######################################################################
            # TODO:                                                               #
            # Compute the l2 distance between the ith test point and all training #
            # points, and store the result in dists[i, :].                        #
            # Do not use np.linalg.norm().                                        #
            #######################################################################
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            dists[i, :] = np.sqrt(np.sum(np.square(X[i,:] - self.X_train), axis = 1))

            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        return dists

    def compute_distances_no_loops(self, X):
        """
        Compute the distance between each test point in X and each training point
        in self.X_train using no explicit loops.

        Input / Output: Same as compute_distances_two_loops
        """
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train))
        #########################################################################
        # TODO:                                                                 #
        # Compute the l2 distance between all test points and all training      #
        # points without using any explicit loops, and store the result in      #
        # dists.                                                                #
        #                                                                       #
        # You should implement this function using only basic array operations; #
        # in particular you should not use functions from scipy,                #
        # nor use np.linalg.norm().                                             #
        #                                                                       #
        # HINT: Try to formulate the l2 distance using matrix multiplication    #
        #       and two broadcast sums.                                         #
        #########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

       	dists = np.sqrt( (X**2).sum(axis=1)[:, np.newaxis] + (self.X_train**2).sum(axis=1) - 2*X.dot(self.X_train.T))

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        return dists

    def predict_labels(self, dists, k=1):
        """
        Given a matrix of distances between test points and training points,
        predict a label for each test point.

        Inputs:
        - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
          gives the distance betwen the ith test point and the jth training point.

        Returns:
        - y: A numpy array of shape (num_test,) containing predicted labels for the
          test data, where y[i] is the predicted label for the test point X[i].
        """
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            # A list of length k storing the labels of the k nearest neighbors to
            # the ith test point.
            closest_y = []
            #########################################################################
            # TODO:                                                                 #
            # Use the distance matrix to find the k nearest neighbors of the ith    #
            # testing point, and use self.y_train to find the labels of these       #
            # neighbors. Store these labels in closest_y.                           #
            # Hint: Look up the function numpy.argsort.                             #
            #########################################################################
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            closest_y = self.y_train[np.argsort(dists)[i,:k]]

            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
            #########################################################################
            # TODO:                                                                 #
            # Now that you have found the labels of the k nearest neighbors, you    #
            # need to find the most common label in the list closest_y of labels.   #
            # Store this label in y_pred[i]. Break ties by choosing the smaller     #
            # label.                                                                #
            #########################################################################
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            indexs, counts = np.unique(closest_y, return_counts = True)
            y_pred[i] = indexs[np.argmax(counts)]
            
            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        return y_pred

    
```
</details>

---

### Hyper Parameters

모델을 만들거나 학습을 시작할 때, 사용자가 직접 세팅해주는 값 혹은 설정을 뜻한다.

---
#### - Hyperparameters in KNN

KNN에서의 hyperparameter는 K값과, distance metric을 선택해 주어야 한다.

K와 distance metric의 변화에 따른 KNN의 변화는 아래와 같다.

<details><summary> K </summary>
       
![07.png](img/07.png)
        
</details>

<details><summary> Distance metric </summary>
        
![08.png](img/08.png)
        
</details>
    
---
#### - Hyperparameter tuning
Hyperparameter는 **problem-dependent**하다. 따라서 좋은 hyperparameter를 고르기 위해서는 학습을 시도해 본 뒤 판단할 수 밖에 없다. 좋은 hyperparameter를 선택하는 과정을 hyperparameter tuning 이라고 한다.


Hyperparameter tuning에는 주로 두가지 방법이 쓰인다. 


- Split data into **Train**, **Val**, and **Test**

![09.png](img/09.png)
<br>
데이터를 Train, Validation, Test 데이터로 나누어 사용한다. 학습을 위한 데이터로 Train 데이터를 사용하며, 학습과정을 확인하기 위해 중간중간 Validation 데이터를 사용하여 테스트 한다. 학습이 완료되면, Test 데이터를 이용하여 최종 평가를 한다.

<br><br>

- **Cross-Validation** : Split data into folds, try each fold as validation and average the results. Useful for small datasets.

![10.png](img/10.png)

두번째 방법은 Cross-Validation 방법이다. 데이터를 모으기 힘들어 데이터셋이 적은 경우 주로 사용하며, 데이터를 fold로 쪼개어 각 fold가 validation 역할을 돌아가며 학습하는 방법이다.

![11.png](img/11.png)

---

<details><summary> 2020 cs231n assignmnet1 Cross-Validation python code <br>
    [assignment1\knn.ipynb]</summary>
    
```python
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []

X_train_folds = np.array_split(X_train, num_folds)
y_train_folds = np.array_split(y_train, num_folds)

k_to_accuracies = {}


for k in k_choices:
    if k not in k_to_accuracies:
        k_to_accuracies[k] = []
    
    for i in range(num_folds):
        X_train_fold = np.concatenate([X_fold for fold_num,X_fold in enumerate(X_train_folds) if fold_num != i])
        y_train_fold = np.concatenate([y_fold for fold_num,y_fold in enumerate(y_train_folds) if fold_num != i])
        X_valid_fold = X_train_folds[i]
        y_valid_fold = y_train_folds[i]
        num_valid = X_valid_fold.shape[0]
        
        classifier.train(X_train_fold, y_train_fold)
        y_valid_pred = classifier.predict(X_valid_fold, k=k)
        num_correct = np.sum(y_valid_pred == y_valid_fold)
        
        accuracy = float(num_correct) / num_valid
        
        k_to_accuracies[k].append(accuracy)
    

# Print out the computed accuracies
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))
```
</details>

---


### Problem of k-Nearest Neighbor

KNN은 아래와 같은 문제로 사용하지 않는다.

- Distance metrics on pixels are not informative
- Very slow at test time
- Curse of dimensionality

***==> Never Used***

---

## Linear Classifier
One of the simplest example of Parametric model

### - Parametric Approach

    어떠한 목적을 잘 할 수 있는 함수 f 와, 그 파라미터 W가 존재할 것이다 라는 가정하에 사용하는 접근법이다.
    이때 f 를 목적함수(objective function) 이라고 한다.
    목적함수 f가 1차 이하의 다항식으로 구성되어 있으면 linear 하다고 한다.


![12.png](img/12.png)

---

### - Interpreting a Linear Classifier
- **Algebraic Viewpoint**
![13.png](img/13.png)

- **Visual Viewpoint**
![14.png](img/14.png)
- **Geometric Viewpoint**
![15.png](img/15.png)

---

## - Loss function
    목적함수 f 를 평가하기위한 함수.
    0에 가까울수록 (낮을 수록) 예측을 잘한다는 의미

$$ L = \frac{1}{N}\sum_{i}{L_i(s_i, y_i )} $$

$$ s = f(x_i ; W) $$
- **Multiclass SVM loss** : 

    $ L_i = \sum_{{j}\neq{y_i}}{\max(0, {s_j-s_{y_i}+ \Delta})} $,
    ($\Delta$ : margin)
    
    margin 의 역할 : 정답을 잘 맞췄더라도, 정답의 Score 값($s_{y_i}$)이 다른 Score값($s_j$) 보다 margin 이상 크지 않으면 loss 를 발생시켜 학습할 수 있도록 해준다. 그 결과 좀더 안정적인 classifier가 될 수 있다.
    
<details><summary> bias, margin </summary>
    
![21.png](img/21.png)
    
</details>
    
    
<br>

- **Softmax Classifier** : 
    - Softmax function : $ P(Y=k|X=x_i) = \frac{e^{s_k}}{\sum_{j}{e^{s_j}}} $
    <br> : score 를 확률변수로 변환하는 역할
    
    <br>
    - cross-entropy loss : $L_i = -logP(Y=y_i|X=x_i)$
    
    $ L_i = -log( \frac{e^{f_{y_i}}}{\sum_{j}{e^{f_j}}}) = -f_{y_i} + log \sum_{j}{e^{f_j}}$

<br>
<details><summary> How to calculate SVM and Softmax loss </summary>

- **SVM Loss**
    
![16.png](img/16.png)
    
- **Softmax loss**

![17.png](img/17.png)

</details>

---

### - Regularization

loss function이 작아지는 방향으로 학습을 진행하면 특정 가중치가 너무 큰 값을 가질 수 있다. 이런 경우 train data에 대해서만 잘 작동하고, 일반적으로 성능이 떨어진다.
이런경우를 overfitting 되었다고 한다.

<img src="img/18.png" alt="Drawing" style="width: 400px;"/>

overfitting을 방지하는 방법으로, loss 함수에 페널티를 부여하여 특정 가중치만 너무 커지지 않도록 한다. 이때의 페널티를 regularization 이라고 한다.

- $L(W) = \frac{1}{N} \sum^{N}_{i=1}{L_i(f(x_i, W),y_i)} + \lambda R(W) $
    - regularization strength : $\lambda$
    - regularization : $R(W)$

#### - Regularization 종류

- L2 regularization : $R(W) = \sum_k \sum_l W^{2}_{k,l}$
- L1 regularization : $R(W) = \sum_k \sum_l |W_{k,l}|$
- Elastic net(L1+L2) : $R(W) = \sum_k \sum_l {\alpha} W^{2}_{k,l} + |W_{k,l}|$
- Dropout
- Batch normalization
- Stochastic depth, fractional pooling, etc

---

## Optimization (최적화)

목적 함수에 맞는 parameter를 찾는 과정

### - Gradient decent(경사하강법)

$L(W)$ 를 최소화 하는 $W$를 찾기 위해, $W$의 값을 **neagtive gradient** 방향으로 update하는 방법

- $W:= W - \alpha\frac{\partial L(W)}{\partial W}$
    ($\alpha$ : learning rate)

#### - Stochastic Gradient Descent(SGD)

    학습데이터의 개수(N)가 많아지면, L(W)를 구하는데 시간이 많이 걸림.
    이러한 단점을 보안하기위해, 데이터를 minibatch로 나누어 N 사이즈를 줄여 학습.

#### - learning rate
    gradient decent를 사용할때 설정해야하는 hyperparameter로 적당한 크기의 learning rate를 찾아야 한다. 크기가 너무 큰 경우 발산할 가능성이 있고, 너무 작은 경우 학습하는데 오랜 시간이 걸린다. 또한 L(W)가 convex 하지 못한경우 local minimum에 빠질 수 있다.
   
![19.png](img/19.png)

---

In [22]:
class Linear_classifier(object):
    def __init__(self):
        self.W = None
        
    def train(self, X, y, learning_rate=1e-3, reg=1e-5, num_iter=100,
             batch_size=200, verbose=False):
        
        num_train, dim = X.shape
        num_classes = np.max(y) + 1
        
        if self.W is None:
            self.W = 0.001 * np.random.randn(dim, num_classes)
            
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
            
            batch_indexs = np.random.choice(num_train,batch_size)
            X_batch = X[batch_indexs]
            y_batch = y[batch_indexs]
            
            #eveluate loss and gradient
            loss, grad = self.loss(X_batch, y_batch, reg)
            loss_history.append(loss)
            
            # update W
            self.W -= learning_rate * grad
            
            if verbose and it%100 == 0:
                print('iteration %d / %d : loss %f' %(it, num_iters, loss))
        
        return loss_history
    
    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        scores = X.dot(self.W)
        y_pred = np.argmax(scores, axis=1)
        
        return y_pred
    
    def loss(self, X_batch, y_batch, reg):
        pass
    
class LinearSVM(Linear_classifier):
    
    def loss(self, X_batch, y_batch, reg):
        Loss = 0.0
        dW = np.zeros_like(self.W)
        
        num_train = X_batch.shape[0]
        
        # calculate svm loss
        scores = X_batch.dot(self.W)
        correct_class_scores = scores[range(num_train), y_batch][:, np.newaxis] # (N,1) shape
        
        margin = np.maximum(0, scores - correct_class_scores + 1)
        margin[range(num_train),y_batch] = 0
        
        Loss = np.sum(margin)
        Loss /= num_train
        Loss += reg*np.sum(self.W*self.W)
        
        # calculate svm gradient
        margin_count = np.zeros(scores.shape)
        margin_count[margin>0] = 1
        margin_count[range(num_train), y_batch] = -np.sum(margin_count, axis=1)
        
        dW = X_batch.T.dot(margin_count)
        
        dW /= num_train
        dW += 2*reg*self.W
        
        return Loss, dW
        
class Softmax(Linear_classifier):
    
    def loss(self, X_batch, y_batch, reg):
        Loss = 0.0
        dW = np.zeros_like(self.W)
        
        num_train = X_batch.shape[0]
        
        # calculate cross-entropy loss
        scores = X_batch.dot(self.W)
        softmaxs = np.exp(scores) / np.sum(np.exp(scores), axis=1, keepdims=True)
        Loss = np.sum( -np.log(softmaxs[range(num_train),y_batch]))
        Loss /= num_train
        Loss += reg *np.sum(self.W * self.W)
        
        # calculate gradients
        softmaxs[range(num_train),y_batch] -= 1
        dW = X_batch.dot(softmaxs)
        dW /= num_train
        dW += reg*2*self.W
        
        return Loss, dW

### Calculate SVM loss gradient
- $s_j = w_j x_i$
- $s_{y_i} = w_{y_i} x_i$
- $L_i = \sum_{j\neq y_i}{max(0,s_j - s_{y_i}+\Delta)}$
에서
<br><br>


$\nabla_{w_j}{L_i} = \nabla_{w_j}\sum_{j\neq y_i}{max(0,s_j - s_{y_i}+\Delta)}= 1(s_j - s_{y_i} + \Delta > 0) \nabla_{w_j}(s_j - s_{y_i} + \Delta) $

$\nabla_{w_j}{s_j} = \nabla_{w_j}{w_j x_i} = x_i$ 이므로

$\therefore \nabla_{w_j}{L_i} =  1(s_j - s_{y_i} + \Delta > 0) x_i$

마찬가지로, 


$\nabla_{w_{y_i}}{L_i} = \nabla_{w_{y_i}}\sum_{j\neq y_i}{max(0,s_j -s_{y_i}+\Delta)}= \sum_{j \neq y_i}{(1 \cdot (s_j - s_{y_i} + \Delta > 0)) \cdot \nabla_{w_{y_i}}(s_j - s_{y_i} + \Delta)} $

$\nabla_{w_{y_i}}{s_{y_i}} = \nabla_{w_{y_i}}{w_{y_i} x_i} = x_i$ 이므로

$\therefore \nabla_{w_{y_i}}{L_i} = -x_i \cdot \sum_{j \neq y_i}{(1 \cdot (s_j - s_{y_i} + \Delta > 0))}$

---


### Calculate cross entropy loss gradient
- $L = -s_{y} + ln(\sum_{j}{e^{s_j}}) $ 에서

$\nabla_{s_i}{L} = -\nabla_{s_i}s_{y} + \nabla_{s_i}ln(\sum_{j}{e^{s_j}})$

$= \frac{\nabla_{s_i}(\sum_{j}{e^{s_j}})}{\sum_{j}{e^{s_j}}} - \nabla_{s_i}{s_y} $

$= \frac{e_{s_i}}{\sum_{j}{e^{s_j}}} - \nabla_{s_i}{s_y} $

$= softmax_i - \nabla_{s_i}{s_y} $

$=
\begin{cases}
    softmax_i - 1 & {y=i} \\
    softmax_i & \text{otherwise}
\end{cases}
$

Chain rule에 의해

$ \frac{\partial L}{\partial W_j}= \frac{\partial L}{\partial s_i} \times \frac{\partial s_i}{\partial W_j} $

$= x_i \times \begin{cases}
    softmax_i - 1 & {y=i} \\
    softmax_i & \text{otherwise}
\end{cases} $  

### Reference
- [cs231n/classification](cs231n.github.io/)
- [cs231n/linear-classify](https://cs231n.github.io/linear-classify/)
- [cs229 lecture-note3](http://cs229.stanford.edu/notes/cs229-notes3.pdf)